# Importing libraries

In [1]:
import pandas as pd
import numpy as np

# Raw dataset

In [2]:
df = pd.read_csv("../data/rspo/rspo_2024_04_23.csv", sep=";", engine="python", on_bad_lines="skip")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55683 entries, 0 to 55682
Data columns (total 50 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Numer RSPO                        55683 non-null  int64  
 1   REGON                             55683 non-null  object 
 2   NIP                               55683 non-null  object 
 3   Typ                               55683 non-null  object 
 4   Nazwa                             55683 non-null  object 
 5   Kod terytorialny województwo      55683 non-null  object 
 6   Kod terytorialny powiat           55683 non-null  object 
 7   Kod terytorialny gmina            55683 non-null  object 
 8   Kod terytorialny miejscowość      55683 non-null  object 
 9   Kod terytorialny ulica            55683 non-null  object 
 10  Województwo                       55683 non-null  object 
 11  Powiat                            55683 non-null  object 
 12  Gmin

# Removing unnecessary quotient marks

In [3]:
toRemove = [
    "REGON",
    "NIP",
    "Kod terytorialny województwo",
    "Kod terytorialny powiat",
    "Kod terytorialny gmina",
    "Kod terytorialny miejscowość",
    "Kod terytorialny ulica",
    "Numer budynku",
    "Numer lokalu",
    "Kod pocztowy",
    "Telefon",
    "Faks",
    "REGON organu prowadzącego",
    "NIP organu prowadzącego"
]

In [4]:
def removeQuotientMark(text: str):
    if pd.isna(text):
        return text
    
    text = text.lstrip("=\"").rstrip("\"")

    if text == "":
        return np.nan
    
    return text

for col in toRemove:
    df[col] = df[col].transform(removeQuotientMark)

# Replacing empty string to NaN values

In [5]:
df['Imię i nazwisko dyrektora'] = df['Imię i nazwisko dyrektora'].transform(lambda text: np.nan if text == ' ' else text)

# Drop empty column

In [6]:
df.drop(['Data likwidacji'], axis=1, inplace=True)

# Lowercase vivoderships

In [7]:
df['Województwo organu prowadzącego'] = df['Województwo organu prowadzącego'].str.lower()
df['Województwo organu prowadzącego'] = df['Województwo organu prowadzącego'].transform(lambda text: np.nan if pd.isna(text) else text[0].upper() + text[1:])

# Convert colum names

In [8]:
def convertColumns(columns: list):
    # lowercase
    columns = [col.lower() for col in columns]

    # remove polish diacritical characters
    converter = {polish: normal for polish, normal in zip(list("ąęćżźśńłó"), list("aeczzsnlo"))}

    for i in range(len(columns)):
        newCol = ""
        for char in columns[i]:
            if char in converter.keys():
                newCol += converter[char]
            else:
                newCol += char
        columns[i] = newCol

    # remove spaces
    columns = ["_".join(col.split()) for col in columns]

    # remove dasches
    columns = [col.replace("-", "_") for col in columns]

    return columns


columns = convertColumns(df.columns.to_list())

In [9]:
df.columns = columns

# Save changes

In [10]:
df.to_csv('../data/rspo/rspo_2024_04_23_clean.csv', index=False, sep=";")